In [11]:
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import random
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import  ReLU
from tensorflow.keras.layers import GlobalAvgPool2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import  Concatenate
from tensorflow.keras.layers import  Input
from tensorflow.keras.layers import  AvgPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
image_generator = ImageDataGenerator(rescale=1./255)
train_loader = image_generator.flow_from_directory(
    directory=r"/content/drive/MyDrive/Colab Notebooks/Data/train",
    target_size=(224, 224),
    batch_size=50,
    shuffle=True
)
test_loader = image_generator.flow_from_directory(
    directory=r"/content/drive/MyDrive/Colab Notebooks/Data/test",
    target_size=(224, 224),
    batch_size=50,
    shuffle=True
)
valid_loader = image_generator.flow_from_directory(
    directory=r"/content/drive/MyDrive/Colab Notebooks/Data/valid",
    target_size=(224, 224),
    batch_size=50,
    shuffle=True
)

Found 613 images belonging to 4 classes.
Found 315 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


In [4]:
idx_to_class = {val:key for key, val in dict(train_loader.class_indices).items()}
idx_to_class

{0: 'adenocarcinoma',
 1: 'largecellcarcinoma',
 2: 'normal',
 3: 'squamouscellcarcinoma'}

In [18]:

import tensorflow.keras.backend as K

def bn_rl_conv(x, f, k=1, s=1, p='same'):
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = Conv2D(f, k, strides=s, padding=p)(x)
    return x

def dense_block(tensor, r):
    for _ in range(r):

        x = bn_rl_conv(tensor, 32)
        x = bn_rl_conv(x, 32, 3)
        tensor = Concatenate()([tensor, x])
    return tensor

def transition_block(x):

    x = bn_rl_conv(x, K.int_shape(x)[-1] // 2)
    x = AvgPool2D(2, strides=2, padding='same')(x)
    return x

repetitions = [3, 6, 12, 8]
input_shape = (224, 224, 3)


input = Input(shape=input_shape)


x = Conv2D(32, 7, strides=2, padding='same')(input)  # Reduced number of filters
x = MaxPooling2D(3, strides=2, padding='same')(x)


for r in repetitions:
    d = dense_block(x, r)
    x = transition_block(d)


x = GlobalAvgPool2D()(d)
output = Dense(4, activation='softmax')(x)


model = Model(input, output)



model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



model.save("lungcancer2003.h5")


batch_size = 8


model_train_history = model.fit(
    x=train_loader,
    steps_per_epoch=9,
    epochs=100,
    validation_data=test_loader,
    validation_steps=7,
    batch_size=batch_size
)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Epoch 1/100
9/9 [==============================] - 481s 44s/step - loss: 1.1750 - accuracy: 0.5012 - val_loss: 1.2794 - val_accuracy: 0.4381
Epoch 2/100
9/9 [==============================] - 21s 2s/step - loss: 1.1264 - accuracy: 0.4911 - val_loss: 1.2777 - val_accuracy: 0.3810
Epoch 3/100
9/9 [==============================] - 8s 897ms/step - loss: 0.9553 - accuracy: 0.5545 - val_loss: 1.2832 - val_accuracy: 0.3810
Epoch 4/100
9/9 [==============================] - 7s 764ms/step - loss: 0.8557 - accuracy: 0.5822 - val_loss: 1.2866 - val_accuracy: 0.3810
Epoch 5/100
9/9 [==============================] - 7s 764ms/step - loss: 0.8172 - accuracy: 0.6178 - val_loss: 1.3030 - val_accuracy: 0.3810
Epoch 6/100
9/9 [==============================] - 9s 1s/step - loss: 0.7173 - accuracy: 0.6778 - val_loss: 1.3718 - val_accuracy: 0.3619
Epoch 7/100
9/9 [==============================] - 6s 725ms/step - loss: 0.7134 - accuracy: 0.6755 - val_loss: 1.5017 - val_accuracy: 0.2857
Epoch 8/100
9/9 [=